# Exploratory Data Analysis

Exploratory data analysis (EDA) is performed on the datasat to investigate features. Firstly the target feature is plot as pie chart to illustrate the distrubtion of passengers that were transported or not.